# 라이브러리 호출

In [407]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re

plt.rc('font', family='nanum')
sns.set(font="Malgun Gothic",#"NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')   

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
import catboost as cbt


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

import DataAnalysis_ksw as da
import Util_for_sklearn_ksw as ufs


# 데이터 불러오기

In [426]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [427]:
train.drop('Unnamed: 0',axis=1,inplace=True)
train.head()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,rank,genre_rank,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,96,0.0,0,91,2,23398,134,11,0,0,1
1,113,837969.0,2,262,4,233211,134,9,1,0,0
2,108,75740.0,1,288,3,781516,134,8,0,0,1
3,87,4598985.0,1,182,3,38809,134,5,0,0,1
4,124,372863.0,2,319,3,2971475,134,12,0,0,1


In [428]:
test.drop('Unnamed: 0',axis=1,inplace=True)
test.head()

,title,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,rank,genre_rank,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,용서는 없다,125.0,300529.00,2.0,304.0,3.0,132.0,12,0,0,1
1,아빠가 여자를 좋아해,113.0,342700.25,4.0,275.0,3.0,144.0,5,0,0,0
2,의형제,116.0,691342.00,2.0,408.0,2.0,144.0,11,1,0,0
3,맨발의 꿈,121.0,0.00,0.0,329.0,2.0,144.0,8,0,1,0
4,더 킥,105.0,0.00,0.0,18.0,7.0,144.0,11,0,0,0


In [429]:
test_pred = test.drop('title',axis=1)

# 왜도 첨도 확인

In [351]:
print(train['dir_prev_bfnum'].skew())
print(train['box_off_num'].skew())
print(train['num_actor'] .skew())
print(train['num_staff'].skew())
###
print(train['rank'].skew())
print(train['time'].skew())
print(train['genre_rank'].skew())

6.1889700504395515
4.332117702173375
2.950667806809796
1.2334523329928717
-0.31717156801654073
0.34564875244035936
-0.37404738220949435


# 스케일링
- dir_prev_bfnum
- box_off_num
- num_actor
- num_staff

-> box_off_num, num_actor 제일 큼

## 로그변환

In [430]:
#train['dir_prev_bfnum'] = np.log1p(train['dir_prev_bfnum'])
train['box_off_num'] = np.log1p(train['box_off_num'])
train['num_actor'] = np.log1p(train['num_actor'])
#train['num_staff'] = np.log1p(train['num_staff'])
###################
#test_pred['dir_prev_bfnum'] = np.log1p(test_pred['dir_prev_bfnum'])
test_pred['num_actor'] = np.log1p(test_pred['num_actor'])
#test_pred['num_staff'] = np.log1p(test_pred['num_staff'])

## 데이터 분리

In [431]:
x = train.drop(['box_off_num'],axis=1)
y = train['box_off_num']

In [413]:
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.3,random_state = 42)

In [414]:
from sklearn.preprocessing import StandardScaler

sclaer = StandardScaler()
x_train_s = sclaer.fit_transform(x_train)
x_test_s = sclaer.transform(x_val)

# 모델링

In [425]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 42)

## RandomForest

### giridsearch

In [422]:
params_rf = {'max_depth': range(1,51,10), 'min_samples_leaf': range(1,101,25), 'n_estimators': range(10,101,20)}
m_rf = GridSearchCV(RandomForestRegressor(random_state = 42),params_rf,cv=10,verbose=3)
m_rf.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.404 total time=   0.0s
[CV 2/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.763 total time=   0.0s
[CV 3/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.739 total time=   0.0s
[CV 4/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.659 total time=   0.0s
[CV 5/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.572 total time=   0.0s
[CV 6/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.661 total time=   0.0s
[CV 7/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.670 total time=   0.0s
[CV 8/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.658 total time=   0.0s
[CV 9/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score=0.729 total time=   0.0s
[CV 10/10] END max_depth=1, min_samples_leaf=1, n_estimators=10;, score

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': range(1, 51, 10),
                         'min_samples_leaf': range(1, 101, 25),
                         'n_estimators': range(10, 101, 20)},
             verbose=3)

In [423]:
m_rf.best_params_

{'max_depth': 31, 'min_samples_leaf': 1, 'n_estimators': 50}

In [416]:
pred_rf = m_rf.predict(x_val)

In [417]:
ufs.Regressor_report(y_val,pred_rf)

MAE :  1.2291143199603352
MAPE :  0.19038626251577564
RMSE :  1.698611833317012


In [418]:
ufs.Regressor_report(np.expm1(y_val),np.expm1(pred_rf))

MAE :  498117.2150339268
MAPE :  6.516221019749363
RMSE :  1382200.2523607258


### Kfold

In [432]:
rf_pred = np.zeros((test.shape[0]))
rmse_list = []

m_rf_k = RandomForestRegressor(max_depth=31, min_samples_leaf=1,n_estimators=50 ,random_state = 42)

for tr_idx,val_idx in kf.split(x,y):
    tr_x,tr_y = x.iloc[tr_idx],y.iloc[tr_idx]
    val_x,val_y = x.iloc[val_idx],y.iloc[val_idx]
    
    m_rf_k.fit(tr_x,tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in m_rf_k.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in m_rf_k.predict(test_pred)])
    rmse = np.sqrt(ufs.mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    rf_pred += (sub_pred / 10)

In [433]:
np.mean(rmse_list)

858762.6040286513

## CatBoost

In [398]:
params_cbt =  {'depth': [5], 'iterations': [100], 'learning_rate': [0.04]}
model_cbt = GridSearchCV(cbt.CatBoostRegressor(),params_cbt,cv=10,verbose=3)
model_cbt.fit(x_train,y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
0:	learn: 1934191.3700651	total: 1.15ms	remaining: 114ms
1:	learn: 1906371.6098799	total: 2.04ms	remaining: 100ms
2:	learn: 1880066.4004807	total: 2.87ms	remaining: 92.8ms
3:	learn: 1851366.4302695	total: 3.72ms	remaining: 89.2ms
4:	learn: 1824327.6091291	total: 4.58ms	remaining: 86.9ms
5:	learn: 1799309.7262671	total: 5.46ms	remaining: 85.6ms
6:	learn: 1772856.1713121	total: 6.33ms	remaining: 84.1ms
7:	learn: 1746037.7805671	total: 7.23ms	remaining: 83.1ms
8:	learn: 1717727.5294454	total: 8.17ms	remaining: 82.6ms
9:	learn: 1700914.1596220	total: 9.81ms	remaining: 88.3ms
10:	learn: 1678061.2382734	total: 10.9ms	remaining: 88.6ms
11:	learn: 1651802.0886228	total: 11.9ms	remaining: 87.3ms
12:	learn: 1631823.7338163	total: 12.8ms	remaining: 85.7ms
13:	learn: 1613072.6342135	total: 13.6ms	remaining: 83.7ms
14:	learn: 1591689.5450651	total: 14.5ms	remaining: 82ms
15:	learn: 1574471.8697356	total: 15.3ms	remaining: 80.6ms
16:	learn

GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001EF1CA7F940>,
             param_grid={'depth': [5], 'iterations': [100],
                         'learning_rate': [0.04]},
             verbose=3)

In [399]:
pred_cbt = model_cbt.predict(x_val)

In [400]:
ufs.Regressor_report(y_val,pred_cbt)

MAE :  618635.1958917223
MAPE :  527.4453100219299
RMSE :  1297311.7150591135


In [401]:
ufs.Regressor_report(np.expm1(y_val),np.expm1(pred_cbt))

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: overflow encountered in expm1
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\User\AppData\Local\Temp\ipykernel_5100\1131518696.py:1: RuntimeWarning: overflow encountered in expm1
  ufs.Regressor_report(np.expm1(y_val),np.expm1(pred_cbt))


ValueError: Input contains infinity or a value too large for dtype('float64').

## Voting

In [370]:
model_vot = GridSearchCV(VotingRegressor([('rf',m_rf),('cbt',model_cbt)]),dict(),cv=10,verbose=3)
model_vot.fit(x_train,y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.784 total time=   0.0s
[CV 2/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.810 total time=   0.0s
[CV 3/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.727 total time=   0.0s
[CV 4/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.723 total time=   0.0s
[CV 5/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.713 total time=   0.0s
[CV 6/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.783 total time=   0.0s
[CV 7/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.727 total time=   0.0s
[CV 8/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.786 total time=   0.0s
[CV 9/10] END max_depth=8, min_samples_leaf=20, n_estimators=10;, score=0.641 total time=   0.0s
[CV 1

GridSearchCV(cv=10,
             estimator=VotingRegressor(estimators=[('rf',
                                                    GridSearchCV(cv=10,
                                                                 estimator=RandomForestRegressor(),
                                                                 param_grid={'max_depth': [8],
                                                                             'min_samples_leaf': [20],
                                                                             'n_estimators': [10]},
                                                                 verbose=3)),
                                                   ('cbt',
                                                    GridSearchCV(cv=10,
                                                                 estimator=<catboost.core.CatBoostRegressor object at 0x000001EF19901730>,
                                                                 param_grid={'depth': [5],
                                                                             'iterations': [100],
                                                                             'learning_rate': [0.04]},
                                                                 verbose=3))]),
             param_grid={}, verbose=3)

In [371]:
pred_vot = model_vot.predict(x_val)

In [372]:
ufs.Regressor_report(y_val,pred_vot)

MAE :  1.0334199260514807
MAPE :  0.1361434575980676
RMSE :  1.5350910216856761


In [373]:
ufs.Regressor_report(np.expm1(y_val),np.expm1(pred_vot))

MAE :  451764.0444319319
MAPE :  11.81864823063392
RMSE :  1445568.156954571


# 테스트 데이터

## 모델별 결과

In [434]:
sub_rf = pd.DataFrame({'title':test['title'],
                        'box_off_num' :rf_pred})

sub_rf.set_index('title',inplace=True)

sub_rf.to_csv('rf.csv')

## 로그변환 모델별 결과

In [374]:
pred_vot_test = np.expm1(model_vot.predict(test_pred))
pred_rf_test = np.expm1(m_rf.predict(test_pred))
pred_cbt_test = np.expm1(model_cbt.predict(test_pred))

In [378]:
sub_vot = pd.DataFrame({'title':test['title'],
                        'box_off_num' : pred_vot_test})
sub_rf = pd.DataFrame({'title':test['title'],
                        'box_off_num' :pred_rf_test})
sub_cbt = pd.DataFrame({'title':test['title'],
                        'box_off_num' :pred_cbt_test})

sub_vot.set_index('title',inplace=True)
sub_rf.set_index('title',inplace=True)
sub_cbt.set_index('title',inplace=True)

In [379]:
sub_vot.to_csv('vot.csv')
sub_rf.to_csv('rf.csv')
sub_cbt.to_csv('cbt.csv')

## 블렌딩 결과

In [404]:
pred_test = (m_rf.predict(test_pred) + model_cbt.predict(test_pred))/2

sub = pd.DataFrame({'title':test['title'],
                        'box_off_num' :pred_test})

sub.set_index('title',inplace=True)

In [405]:
sub.shape

(243, 1)

In [406]:
sub.to_csv('sub.csv')